In [12]:
import pandas as pd
import tensorflow as tf

df = pd.read_excel('./result/newscore!!!.xlsx')
# df2 = pd.read_excel('./result/score.xlsx')
# df = df1.append(df2)
df

,index,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011
0,display,100,100,100,100,100,99,100,98,95,89,94
1,assembly,100,100,99,98,96,95,97,85,99,92,97
2,plant,100,86,0,0,51,0,0,0,0,0,10
3,system,100,100,100,100,100,100,100,100,100,100,100
4,device,99,99,100,99,99,100,100,100,100,100,100
...,...,...,...,...,...,...,...,...,...,...,...,...
693,adapted,0,0,0,0,0,0,0,0,0,0,29
694,cutting,0,0,0,0,0,0,0,0,0,0,26
695,coil,0,0,0,0,0,0,0,0,0,0,12
696,elongate,0,0,0,0,0,0,0,0,0,0,4


In [2]:
import numpy as np
years = ['2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']

dateset = np.zeros((1,7))

for i in range(5):
    print(df[years[i: i+7]].to_numpy().shape)
    dateset = np.concatenate((dateset, df[years[i: i+7]].to_numpy()), axis = 0)
    
dateset = dateset[1:] / 100


(698, 7)
(698, 7)
(698, 7)
(698, 7)
(698, 7)


In [3]:
np.random.shuffle(dateset)
dateset

array([[0.  , 0.05, 0.35, ..., 0.04, 0.  , 0.21],
       [0.  , 0.34, 0.  , ..., 0.21, 0.06, 0.57],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.12, 0.  , ..., 0.  , 0.  , 0.  ],
       [0.28, 0.3 , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]])

In [4]:
train_data = dateset[:,:-1].reshape(-1,6,1)
label_data = dateset[:,-1]
print(train_data.shape)
print(label_data.shape)


(3490, 6, 1)
(3490,)


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)

In [22]:


inputs = tf.keras.Input(shape=(6, 1))
# inputs = tf.random.uniform(shape=[5,5,1])
gru1 = tf.keras.layers.GRU(6, return_sequences=True)(inputs)
# gru1 = tf.keras.layers.LayerNormalization()(gru1)
gru2 = tf.keras.layers.GRU(12, return_sequences=True)(gru1)
# gru2 = tf.keras.layers.LayerNormalization()(gru2)
gru3 = tf.keras.layers.GRU(6, return_state=True )(gru2)
gru3 = tf.keras.layers.LayerNormalization()(gru3[-1])
gru3 = tf.keras.layers.Dropout(0.1)(gru3)
d = tf.keras.layers.Dense(32)(gru3)
d = tf.keras.layers.Dense(16)(d)
d = tf.keras.layers.Dense(4)(d)
outputs = tf.keras.layers.Dense(1)(d)

model = tf.keras.Model(inputs, outputs)

model.summary()

model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.0005), loss='mse',)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 6, 1)]            0         
                                                                 
 gru_3 (GRU)                 (None, 6, 6)              162       
                                                                 
 gru_4 (GRU)                 (None, 6, 12)             720       
                                                                 
 gru_5 (GRU)                 [(None, 6),               360       
                              (None, 6)]                         
                                                                 
 layer_normalization_1 (Laye  (None, 6)                12        
 rNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 6)                 0   

C:\Program Files\Python38\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [31]:

model.load_weights('model/gru')
avg = np.sum(df[['2016','2017','2018']].to_numpy(), axis = 1)/3
input_data = df[['2016','2017','2018','2019','2020','2021']].to_numpy()/100

predicts = model.predict(input_data)*100
inds = np.argsort(predicts, axis = 0)
inds = inds.reshape(-1)

names = df['index'].to_numpy()
for ind in inds:
    if predicts[ind] - avg[ind]> 10 :
        print(names[ind], predicts[ind])


acoustic [10.211865]
shell [10.211865]
kalanchoe [10.211865]
nodes [10.299739]
write [10.305708]
temperature [10.418263]
waveguide [10.515829]
uf [10.515829]
event [10.519024]
background [10.541259]
cooling [10.696478]
coupler [10.820989]
software [10.863321]
intelligence [10.936729]
phase [11.0151005]
measuring [11.0151005]
plants [11.127303]
face [11.253359]
group [11.253359]
preparing [11.570575]
solution [11.743384]
prosthesis [11.743384]
residual [12.053127]
operations [12.101299]
artificial [12.268143]
gel [12.363991]
fuel [12.457797]
subject [12.617606]
blueberry [12.675953]
coleus [12.989002]
defining [13.025758]
product [13.303137]
purple [13.618335]
pha [13.934576]
internal [14.178809]
along [14.344141]
dynamic [14.427796]
fruit [14.570165]
pipe [14.570165]
impatiens [14.889476]
calibration [14.928013]
enhancing [15.209782]
technical [15.431148]
symbol [15.531069]
early [15.853325]
flowering [16.176533]
garden [16.50067]
echinacea [16.825739]
value [16.974268]
freely [17.1517

In [8]:
import pandas as pd
df = pd.read_excel('./result/newscore!!!.xlsx')
df[df['point'] == 'described']

KeyError: 'point'

In [61]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np

model = Word2Vec.load('./model/model1.model')
sims = model.wv.most_similar('plant', topn=10) #0-1 사이의 값으로 나옴
sims

[('tnechdsw’', 0.8796506524085999),
 ('helianthus', 0.8769041299819946),
 ('koncacandy’', 0.8756771683692932),
 ('tnechps’', 0.8754878044128418),
 ('drisstrawsixty’', 0.8748146295547485),
 ('tnechsdl’', 0.8742573857307434),
 ('lilshreckthu’', 0.8722013235092163),
 ('js’', 0.8711698651313782),
 ('abesrpras’', 0.8707885146141052),
 ('verschercru’', 0.8704249858856201)]